In [ ]:
import sys
from shutil import copy2
from imp import find_module

import numpy as np
import pandas as pd
import scipy.optimize as opt

from bikewheelcalc import BicycleWheel, Hub, Rim
import bikewheellib as bl
from doetools import RadialBucklingDOE, TensionBucklingDOE, LateralStiffnessDOE

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

from IPython.display import display
from __future__ import print_function

In [ ]:
# Set default colors
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
                                                    '#9467bd', '#8c564b', '#e377c2', '#7f7f7f',
                                                    '#bcbd22', '#17becf'])

In [ ]:
# Helper functions

rho_rim = 2700.0  # [kg/m^3] Density
rho_spk = 8000.0
E_rim = 69.0e9    # [Pa] Young's modulus
G_rim = 26.0e9    # [Pa] Shear modulus
E_spk = 200.0e9

def create_wheel_obj(f_rim, M, ns, R, r, d, w):
    """Create BicycleWheel object from optimization parameters
         M  = total mass
         ns = number of spokes (arbitrary in continuum model)
         R  = rim radius
         r  = rim cross-section outer radius
         d  = hub diameter
         w  = hub width"""
    
    # rim wall thickness
    t = r - np.sqrt(r**2 - M*f_rim/(2*np.pi**2*R*rho_rim))
    
    ls = np.sqrt((w/2)**2 + (R-d/2)**2)
    ds = 2*np.sqrt(M*(1-f_rim)/(ns*np.pi*ls*rho_spk))
    
    wheel = BicycleWheel()
    wheel.hub = Hub(diam1=d, width1=w/2)
    wheel.rim = Rim(radius=R,
                    area=np.pi*(r**2 - (r-t)**2),
                    I11=np.pi/2*(r**4 - (r-t)**4),
                    I22=np.pi/4*(r**4 - (r-t)**4),
                    I33=np.pi/4*(r**4 - (r-t)**4),
                    Iw=0.0,
                    young_mod=E_rim, shear_mod=G_rim)
    
    wheel.lace_radial(n_spokes=ns, diameter=ds, young_mod=E_spk)

    return wheel

def K_rad(w):
    'Calculate radial stiffness from Hetenyi'
    
    R = w.rim.radius
    EI = w.rim.young_mod * w.rim.I33
    
    k_spk = bl.calc_continuum_stiff(w)
    k_vv = k_spk[1, 1]
    
    e = np.sqrt(R**4*k_vv/EI + 1)
    a = np.sqrt((e-1)/2)
    b = np.sqrt((e+1)/2)
    
    y0 = w.rim.radius**3/(4*a*b*EI)
    
    t1 = 2*a*b/(np.pi*e**2)
    t2 = (b*np.sinh(a*np.pi)*np.cosh(a*np.pi) + a*np.sin(b*np.pi)*np.cos(b*np.pi)) /\
        (e*(np.sinh(a*np.pi)**2 + np.sin(b*np.pi)**2))
        
    return -1.0 / (y0*(t1 - t2))

def f_rim_max(M, R, r):
    'Maximum rim weight fraction (rim cross-section is a solid disk)'
    
    return min(1.0, 2*np.pi**2*R*r**2*rho_rim/M)

def f_obj_Tc(f_rim, M, ns, R, r, d, w):
    'Objective function for buckling tension'
    
    w = create_wheel_obj(f_rim, M, ns, R, r, d, w)
    Tc, n = bl.calc_buckling_tension(w, approx='linear')
    
    return -Tc

def f_obj_Klat(f_rim, M, ns, R, r, d, w):
    'Objective function for lateral stiffness'
    
    w = create_wheel_obj(f_rim, M, ns, R, r, d, w)
    Klat = bl.lateral_stiffness(w)
    
    return -Klat

def f_obj_Pmax(f_rim, M, ns, R, r, d, w):
    'Objective function for radial strength'
    
    w = create_wheel_obj(f_rim, M, ns, R, r, d, w)
    
    Klat = bl.lateral_stiffness(w)
    Krad = K_rad(w)
    Tc, n = bl.calc_buckling_tension(w, approx='linear')
    Ks = w.spokes[0].EA / w.spokes[0].length
    
    # return -Klat*R / (1 + EA/Tc * Klat/Krad)
    return -Klat*R / (1 + Ks*Klat*R*w.spokes[0].n[1]/(Krad*Tc))

## Radial buckling ABAQUS simulations

In [ ]:
M = 0.8
R = 0.3
ns = 36
r = 0.010
d = 0.050
w = 0.050

doe_opts = {'spk_paired': False, 'spk_eltype': 'beam', 'sim_u2': 0.01}
doe_Pc_dir = '../data/doe/doe_optimization_Pc'
doe = RadialBucklingDOE(out_dir=doe_Pc_dir, opts=doe_opts)

for fr in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:

    wheel = create_wheel_obj(fr, M, ns, R, r, d, w)

    Tc_theor = bl.calc_buckling_tension(wheel)[0]

    for Tn in [0.0, 0.10, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
        
        jobname = 'fr{:.1f}_Tn{:.2f}'.format(fr, Tn)
        
        exp_opts = {'jobname': jobname, 'f_rim': fr,
                    'spk_Tn': Tn, 'spk_T': Tn*Tc_theor}

        doe.add_experiment(wheel, opts=exp_opts)

print('\nCreated {0:d} simulations'.format(len(doe.db)))

doe.write_input_files(N_batches=4)
doe.to_csv()

# Copy postprocessing script
copy2(src=find_module('doetools')[1] + '/postproc_rad_buckling.py',
      dst=doe_Pc_dir)

del doe

## Tension buckling ABAQUS simulations

In [ ]:
M = 0.8
R = 0.3
ns = 36
r = 0.010
d = 0.050
w = 0.050

up = 0.01e-3
doe_opts = {'spk_paired': False, 'spk_eltype': 'beam',
            'rim_perturb': [0., up, up, up, up]}
doe_Tc_dir = '../data/doe/doe_optimization_Tc'
doe = TensionBucklingDOE(out_dir=doe_Tc_dir, opts=doe_opts)

for fr in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:

    wheel = create_wheel_obj(fr, M, ns, R, r, d, w)

    jobname = 'fr{:.1f}'.format(fr)

    doe.add_experiment(wheel, opts={'jobname': jobname, 'f_rim': fr})

print('\nCreated {0:d} simulations'.format(len(doe.db)))

doe.write_input_files(N_batches=4)
doe.to_csv()

# Copy postprocessing script
copy2(src=find_module('doetools')[1] + '/postproc_tension.py',
      dst=doe_Tc_dir)

del doe

## Lateral stiffness ABAQUS simulations

In [ ]:
M = 0.8
R = 0.3
ns = 36
r = 0.010
d = 0.050
w = 0.050

up = 0.01e-3
doe_opts = {'spk_paired': False, 'spk_eltype': 'beam', 'spk_T': 0.}

doe_Klat_dir = '../data/doe/doe_optimization_Klat'
doe = LateralStiffnessDOE(out_dir=doe_Klat_dir, opts=doe_opts)

for fr in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:

    wheel = create_wheel_obj(fr, M, ns, R, r, d, w)

    jobname = 'fr{:.1f}'.format(fr)

    doe.add_experiment(wheel, opts={'jobname': jobname, 'f_rim': fr})

print('\nCreated {0:d} simulations'.format(len(doe.db)))

doe.write_input_files(N_batches=4)
doe.to_csv()

# Copy postprocessing script
copy2(src=find_module('doetools')[1] + '/postproc_lat_stiffness.py',
      dst=doe_Klat_dir)

del doe

## Extract results

In [ ]:
# Load Klat database
print('Loading K_lat database...')
doe_K = LateralStiffnessDOE(out_dir=doe_Klat_dir, db_file=doe_Klat_dir+'/_doe_db.csv')
print('Extracting results...')
d_K = doe_K.extract_results()

# Load Tc database
print('\nLoading T_c database...')
doe_T = TensionBucklingDOE(out_dir=doe_Tc_dir, db_file=doe_Tc_dir+'/_doe_db.csv')
print('Extracting results...')
d_T = doe_T.extract_results()

# Load Pc database
print('\nLoading P_c database...')
doe_P = RadialBucklingDOE(out_dir=doe_Pc_dir, db_file=doe_Pc_dir+'/_doe_db.csv')
print('Extracting results...')
d_P = doe_P.extract_results()
print('\nDone')

In [ ]:
R = 0.3
ns = 36
r = 0.010
d = 0.050
w = 0.050

fig, ax = plt.subplots(ncols=3, figsize=(6.5, 2.5))

Masses = [0.1, 0.5, 0.8, 2.0]

cp = sns.color_palette('Reds', len(Masses))

N_pts = 31
print('_'*N_pts)

for j, M in enumerate(Masses):
    fr_max = f_rim_max(M, R, r)
    
    f_rim = np.linspace(0.001, 0.99*fr_max, N_pts)
    Tc = np.zeros(f_rim.shape)
    Pc = np.zeros(f_rim.shape)
    Klat = np.zeros(f_rim.shape)
    nc = np.zeros(f_rim.shape)
    
    for i, f in enumerate(f_rim):
        print('#', end='')
        wheel = create_wheel_obj(f, M, ns, R, r, d, w)
        Tc[i], nc[i] = bl.calc_buckling_tension(wheel)
        Klat[i] = bl.lateral_stiffness(wheel)
        Pc[i] = -f_obj_Pmax(f, M, ns, R, r, d, w)
    
    ax[0].plot(f_rim, Klat/1000, '-', color=cp[j], label='$M=${:.1f}'.format(M))
    ax[0].plot(f_rim[np.argmax(Klat)], np.max(Klat)/1000, 'ko')
    
    ax[1].plot(f_rim, Tc/1000, '-', color=cp[j], label='$M={:.1f}$'.format(M))
    ax[1].plot(f_rim[np.argmax(Tc)], np.max(Tc)/1000, 'ko')
    
    ax[2].plot(f_rim, Pc/1000, '-', color=cp[j])
    ax[2].plot(f_rim[np.argmax(Pc)], np.max(Pc)/1000, 'ko')
    
    print('\n{:.1f} {:.2f} {:.3f} {:.3f} {:.3f}'
          .format(M, fr_max, f_rim[np.argmax(Klat)], f_rim[np.argmax(Tc)], f_rim[np.argmax(Pc)]))

# Plot ABAQUS results
ax[0].plot(doe_K.db['f_rim'], doe_K.db['K_lat_abq']/1000, '*', color=cp[2])

ax[1].plot(doe_T.db['f_rim'], doe_T.db['Tc_nonlin']/1000, '*', color=cp[2])

g = doe_P.db.groupby('f_rim').aggregate([np.mean, np.std])
ax[2].errorbar(g.index, g[('Pc_max', 'mean')]/1000.,
               yerr=g[('Pc_max', 'std')]/1000.,
               fmt='.', color=cp[2], elinewidth=2)
    
ax[0].set_ylim([0., 650.])
ax[0].set_ylabel('Lateral stiffness [N/mm]')

ax[1].set_ylim([0., 10.])
ax[1].set_ylabel('Buckling tension [kN]')

ax[2].set_ylim([0., 20.])
ax[2].set_ylabel('Radial strength [kN]')

for a in ax:
    a.set_xlim([0., 1.])
    a.set_xticks([0., 0.25, 0.5, 0.75, 1.])
    a.set_xticklabels(['0', '', '0.5', '', '1'])
    a.set_xlabel(u'\$f_{rim}\$')

plt.tight_layout()
plt.savefig('../figs/optimization/_python_1D_trends.pdf')

## Scaling with $R$ and $M$

In [ ]:
f_obj = f_obj_Pmax

N_pts = 16

radius = np.logspace(-1, 2, N_pts)
fr_opt_r = np.zeros(radius.shape)
fr_max_r = np.zeros(radius.shape)
F_opt_r = np.zeros(radius.shape)

print('Radius')
print('_'*len(radius))
for i, R in enumerate(radius):
    
    ns = 36
    M = 1.0
    ns = 36
    r = 0.01/0.3 * R
    d = 0.05/0.3 * R
    w = 0.05/0.3 * R
    
    fr_max_r[i] = f_rim_max(M, R, r)

    res = opt.minimize_scalar(lambda x: f_obj(x, M, ns, R, r, d, w),
                              bounds=[0.001, 0.999*fr_max_r[i]], method='bounded',
                              options={'xatol': 1e-5})
    
    print('#', end='')
    fr_opt_r[i] = res.x
    F_opt_r[i] = -f_obj(res.x, M, ns, R, r, d, w)


mass = np.logspace(-2, 1, N_pts)

fr_opt_m = np.zeros(radius.shape)
fr_max_m = np.zeros(radius.shape)
F_opt_m = np.zeros(radius.shape)

print('\nMass')
print('_'*len(mass))
for i, M in enumerate(mass):
    
    ns = 36
    R = 1.0
    ns = 36
    r = 0.01/0.3 * R
    d = 0.05/0.3 * R
    w = 0.05/0.3 * R
    
    fr_max_m[i] = f_rim_max(M, R, r)

    res = opt.minimize_scalar(lambda x: f_obj(x, M, ns, R, r, d, w),
                              bounds=[0.001, 0.999*fr_max_m[i]], method='bounded',
                              options={'xatol': 1e-5})
    
    print('#', end='')
    fr_opt_m[i] = res.x
    F_opt_m[i] = -f_obj(res.x, M, ns, R, r, d, w)

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(5.5, 2.5), sharey=True)

ax[0].loglog(radius[fr_max_r == 1.0], F_opt_r[fr_max_r == 1.0], 'C0o')
ax[0].loglog(radius[fr_max_r != 1.0], F_opt_r[fr_max_r != 1.0], 'C3*')
ax[0].set_ylabel('Radial strength [N]')
ax[0].set_xlabel('Radius [m]')

ax[1].loglog(mass[fr_max_m == 1.0], F_opt_m[fr_max_m == 1.0], 'C0o')
ax[1].loglog(mass[fr_max_m != 1.0], F_opt_m[fr_max_m != 1.0], 'C3*')
ax[1].set_xlabel('Mass [kg]')

ax[0].set_xlim([0.1, 100.])
ax[1].set_xlim([0.01, 10.])

ax[0].set_ylim([1e1, 1e5])

plt.tight_layout()
plt.savefig('../figs/optimization/1D_MR_scaling.svg')